# The QuDev Surface Code

To install the required version of `topological_codes` use
    pip install git+https://github.com/NCCR-SPIN/topological_codes.git

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer
from qiskit.providers.aer.noise.errors import pauli_error
from topological_codes import SurfaceCode, GraphDecoder
import numpy as np
import pickle

The surface code of the ["Realizing Repeated Quantum Error Correction in a Distance-Three Surface Code"](https://arxiv.org/abs/2112.03708) paper by the QuDev lab doesn't use the same conventions for the naming of plaquettes or syndrome measurement as the `SurfaceCode` class of `topological_codes`. We therefore need custom methods to replace the standard ones.

In [2]:
def _get_plaquettes(self):
    assert self.d==3, 'The QuDev convention is not defined d='+str(self.d)+': only d=3 can be used.'
    zplaqs = [[0,3,None,None],[4,7,3,6],[2,4,1,5],[None,None,5,8]]
    xplaqs = [[None,None,2,1],[0,1,4,3],[4,5,8,7],[6,7,None,None]]
    return zplaqs, xplaqs

def syndrome_measurement(self, final=False, barrier=False):
    """
    Application of a syndrome measurement round.
    Args:
        final (bool): If set to true add a reset at the end of each measurement.
        barrier (bool): Boolean denoting whether to include a barrier at the end.
    """
    
    self._resets = False

    num_bits = int((self.d**2 - 1)/2)

    # classical registers for this round
    self.zplaq_bits.append(ClassicalRegister(
        self._num_xy, 'round_' + str(self.T) + '_zplaq_bit'))
    self.xplaq_bits.append(ClassicalRegister(
        self._num_xy, 'round_' + str(self.T) + '_xplaq_bit'))

    for log in ['0', '1']:

        self.circuit[log].add_register(self.zplaq_bits[-1])
        self.circuit[log].add_register(self.xplaq_bits[-1])

        # z plaquette measurement

        self.circuit[log].h(self.zplaq_qubit) # part of ry(pi/2)
        self.circuit[log].x(self.zplaq_qubit) # part of ry(pi/2)

        self.circuit[log].cz(self.code_qubit[0], self.zplaq_qubit[0])
        self.circuit[log].cz(self.code_qubit[4], self.zplaq_qubit[1])
        self.circuit[log].cz(self.code_qubit[2], self.zplaq_qubit[2])

        self.circuit[log].cz(self.code_qubit[3], self.zplaq_qubit[0])
        self.circuit[log].cz(self.code_qubit[7], self.zplaq_qubit[1])
        self.circuit[log].cz(self.code_qubit[4], self.zplaq_qubit[2])

        self.circuit[log].y(self.code_qubit)

        self.circuit[log].cz(self.code_qubit[3], self.zplaq_qubit[1])
        self.circuit[log].cz(self.code_qubit[1], self.zplaq_qubit[2])
        self.circuit[log].cz(self.code_qubit[5], self.zplaq_qubit[3])

        self.circuit[log].cz(self.code_qubit[6], self.zplaq_qubit[1])
        self.circuit[log].cz(self.code_qubit[5], self.zplaq_qubit[2])
        self.circuit[log].cz(self.code_qubit[8], self.zplaq_qubit[3])

        self.circuit[log].x(self.zplaq_qubit) # part of ry(-pi/2)
        self.circuit[log].h(self.zplaq_qubit) # part of ry(-pi/2)

        self.circuit[log].measure(self.zplaq_qubit, self.zplaq_bits[self.T])

        # x plaquette measurement

        self.circuit[log].h(self.xplaq_qubit) # part of ry(pi/2)
        self.circuit[log].x(self.xplaq_qubit) # part of ry(pi/2)

        self.circuit[log].x(self.code_qubit) # part of ry(-pi/2)
        self.circuit[log].h(self.code_qubit) # part of ry(-pi/2)

        self.circuit[log].cz(self.code_qubit[0], self.xplaq_qubit[1])
        self.circuit[log].cz(self.code_qubit[4], self.xplaq_qubit[2]) 
        self.circuit[log].cz(self.code_qubit[6], self.xplaq_qubit[3])

        self.circuit[log].cz(self.code_qubit[1], self.xplaq_qubit[1])
        self.circuit[log].cz(self.code_qubit[5], self.xplaq_qubit[2]) 
        self.circuit[log].cz(self.code_qubit[7], self.xplaq_qubit[3]) 

        self.circuit[log].y(self.code_qubit)

        self.circuit[log].cz(self.code_qubit[2], self.xplaq_qubit[0])
        self.circuit[log].cz(self.code_qubit[4], self.xplaq_qubit[1]) 
        self.circuit[log].cz(self.code_qubit[8], self.xplaq_qubit[2])

        self.circuit[log].cz(self.code_qubit[1], self.xplaq_qubit[0])
        self.circuit[log].cz(self.code_qubit[3], self.xplaq_qubit[1]) 
        self.circuit[log].cz(self.code_qubit[7], self.xplaq_qubit[2])

        self.circuit[log].x(self.code_qubit) # part of ry(-pi/2)
        self.circuit[log].h(self.code_qubit) # part of ry(-pi/2)
        self.circuit[log].x(self.xplaq_qubit) # part of ry(-pi/2)
        self.circuit[log].h(self.xplaq_qubit) # part of ry(-pi/2)

        self.circuit[log].measure(self.xplaq_qubit, self.xplaq_bits[self.T])

    self.T += 1

With these we can construct a class for QuDev surface codes.

In [3]:
class QudevSurfaceCode(SurfaceCode):
    pass

QudevSurfaceCode._get_plaquettes = _get_plaquettes
QudevSurfaceCode.syndrome_measurement = syndrome_measurement

To create a `QudevSurfaceCode` object we need to specify:
* `d` - The code distance (which has to be 3);
* `T` - the number of syndrome measurement rounds (there's currently a bug if this is not even);
* `basis` - Whether to encode the states $|0\rangle$ and $|1\rangle$ (`basis='z'`), or $|+\rangle$ and $|-\rangle$ (`basis='x'`).

Note that qubits and stabilizers are numbered from 1 in the paper but from 0 in this object. So `code_qubit[0]` is qubit D1, `zplaq_qubit[0]` is the auxillary qubit for the stabilizer $S^{Z1}$, and so on.

In [4]:
d = 3
T = 2
basis = 'z'

code = QudevSurfaceCode(d,T,basis=basis)

The code object contains circuits for the two possible logical encoded states: `code.circuit['0']`  `code.circuit['1']`. Note that `'0'` and `'1'` are used even for `basis='x'`.

We can run these on a simulato to see the format of `raw_results` expected when processing results.

In [5]:
job = Aer.get_backend('aer_simulator').run([code.circuit['0'],code.circuit['1']],shots=5)
raw_results = {'0':job.result().get_counts(0), '1':job.result().get_counts(1)}
raw_results

{'0': {'000000110 0000 0000 1010 0000': 1,
  '101110000 0000 0000 0100 0000': 1,
  '101101101 0000 0000 0001 0000': 1,
  '110101011 0000 0000 0111 0000': 1,
  '101101011 0000 0000 0000 0000': 1},
 '1': {'100100010 0000 0000 1010 0000': 1,
  '100111111 0000 0000 0101 0000': 1,
  '100100010 0000 0000 1111 0000': 1,
  '001001001 0000 0000 0011 0000': 1,
  '100111111 0000 0000 1100 0000': 1}}

Here the output bit strings are of the form

    'yxwvutsrq ponm lkji hgfe dcba'

Where
* `dcba` is the result of the first round of Z syndrome measurements (`a` is the result of $S^{Z1}$, `b` is the result of $S^{Z2}$, and so on).
* `hgfe` is the result of the first round of X syndrome measurements.
* `lkji` is the result of the second round of Z syndrome measurements.
* `ponm` is the result of the second round of X syndrome measurements.
* `yxwvutsrq` is the result of a final measurement of all qubits in the basis specified by `basis` (`q` is the result from qubit D1, and so on).

The code object has the method `process_results` to put this into a more useful form for the decoder.

In [6]:
results = code.process_results(raw_results)
results

{'0': {'0 0  0000 0000 0000': 5}, '1': {'1 1  0000 0000 0000': 5}}

These strings are of the form

`'n m  dcba hgfe lkji'`

where
* `dcba` is the result of the first round of relevant syndrome measurements for the given `basis`;
* `hgfe` has the differences between the first and second round;
* `lkji` has the differences between the first and an effective final round extracted from the final code qubit measurements;
* `n` and `m` are the two different measurements of the bare logical value for `basis`.

Decoder objects are constructed for a given code object using `decoder = GraphDecoder(code)`. The process of creating the syndrome graph can take a while, so best to load in a premade one if possible.

In [7]:
try:
    S = pickle.load(open( 'graphs/S'+str(T)+'.p', 'rb' ))
    decoder = GraphDecoder(code, S=S)
except:
    decoder = GraphDecoder(code)

Each edge in the syndrome graph represents a single qubit error that could occur at some point in the circuit. One of the things that the decoder can do is analzse the results to determine the probability of each of these errors.

In [8]:
decoder.get_error_probs(results)

{((1, 0, 3), (1, 1, 3)): 0,
 ((1, 0, 2), (1, 1, 2)): 0,
 ((1, 0, 1), (1, 1, 1)): 0,
 ((1, 0, 0), (1, 1, 0)): 0,
 ((1, 0, 1), (1, 0, 2)): 0,
 ((1, 0, 2), (1, 0, 3)): 0,
 ((1, 0, 1), (1, 1, 2)): 0,
 ((1, 0, 2), (1, 1, 3)): 0,
 ((1, 1, 1), (1, 1, 2)): 0,
 ((1, 0, 0), (1, 0, 1)): 0,
 ((1, 0, 1), (1, 1, 0)): 0,
 ((1, 1, 2), (1, 1, 3)): 0,
 ((1, 1, 0), (1, 1, 1)): 0,
 ((1, 1, 1), (1, 1, 3)): 0,
 ((1, 1, 0), (1, 1, 2)): 0,
 ((1, 1, 3), (1, 2, 3)): 0,
 ((1, 1, 2), (1, 2, 2)): 0,
 ((1, 1, 1), (1, 2, 1)): 0,
 ((1, 1, 0), (1, 2, 0)): 0,
 ((1, 1, 1), (1, 2, 2)): 0,
 ((1, 1, 2), (1, 2, 3)): 0,
 ((1, 2, 1), (1, 2, 2)): 0,
 ((1, 1, 1), (1, 2, 0)): 0,
 ((1, 2, 2), (1, 2, 3)): 0,
 ((1, 2, 0), (1, 2, 1)): 0,
 ((1, 2, 1), (1, 2, 3)): 0,
 ((1, 2, 0), (1, 2, 2)): 0,
 ((1, 0, 3), (1, 0, 3)): 0.0,
 ((1, 0, 1), (1, 0, 1)): 0.0,
 ((1, 0, 2), (1, 0, 2)): 0.0,
 ((1, 1, 3), (1, 1, 3)): 0.0,
 ((1, 1, 1), (1, 1, 1)): 0.0,
 ((1, 1, 2), (1, 1, 2)): 0.0,
 ((1, 0, 0), (1, 0, 0)): 0.0,
 ((1, 1, 0), (1, 1, 0)): 0.0,
 ((1

Here the edges are labelled $((1,t,s),(1,t',s')$. The two sets of coordinates here represent the two syndrome measurements at which the error was detected. For these `t` is the round and `s` is the syndrome.

We can associate differet types of edges with different kinds of noise:
* $((1,t,s),(1,t+1,s))$ due to measurement errors on auxilliary qubits;
* $((1,t,s),(1,t,s'))$ due to an error on the code qubit shared by $s$ and $s'$;
* $((1,t,s),(1,t+1,s'))$ due to an error between the entangling gates while measuring $s$ and $s'$.

In the results above there were no errors, so all probailities are zero. As an example, let's add some measurement errors.

In [9]:
p_meas = 0.01

def noisy_circuit(qc,p_meas):
    'adds a bit flip error before every measurememnt'
    error_qc = QuantumCircuit()
    for qreg in qc.qregs:
        error_qc.add_register(qreg)
    for creg in qc.cregs:
        error_qc.add_register(creg)
    depth = len(qc)
    for j in range(depth):
        if qc.data[j][0].name=='measure':
            qubit = qc.data[j][1]
            error_qc.append(pauli_error([('X', p_meas), ('I', 1 - p_meas)]),qubit)
        error_qc.data.append(qc.data[j])    
    return error_qc

circuits = [noisy_circuit(code.circuit[log], p_meas) for log in ['0', '1']]

job = Aer.get_backend('aer_simulator').run(circuits, shots=1024)
raw_results = {'0':job.result().get_counts(0), '1':job.result().get_counts(1)}

results = code.process_results(raw_results)

error_probs = decoder.get_error_probs(results)

In these results we have the eight errors associated with error probs in the syndrome measurement. These should all be $~0.01$.

In [26]:
done = []
for t in range(T):
    for s in range(4):
        edge = ((1,t,s),(1,t+1,s))
        done.append(edge)
        print(edge,error_probs[edge])

((1, 0, 0), (1, 1, 0)) 0.008885558692194595
((1, 0, 1), (1, 1, 1)) 0.008867698578071515
((1, 0, 2), (1, 1, 2)) 0.010904319809213858
((1, 0, 3), (1, 1, 3)) 0.011895624339183564
((1, 1, 0), (1, 2, 0)) 0.010870378947497095
((1, 1, 1), (1, 2, 1)) 0.007812633341885256
((1, 1, 2), (1, 2, 2)) 0.013918560239978173
((1, 1, 3), (1, 2, 3)) 0.0138682643057908


There are also errors that show up as errors on code qubits, during the final effective syndrome measurement round (done by directly measuring code qubits).

In [27]:
t = T
for s in range(4):
    edge = ((1,t,s),(1,t,s))
    done.append(edge)
    print(edge,error_probs[edge])

((1, 2, 0), (1, 2, 0)) 0.005735444986191884
((1, 2, 1), (1, 2, 1)) 0.017880724864877073
((1, 2, 2), (1, 2, 2)) 0.021433617657113768
((1, 2, 3), (1, 2, 3)) 0.007370427480762276


What remains is either very small (and so just not really an error), or some other final round stuff.

In [28]:
for edge in error_probs:
    prob = round(error_probs[edge],3)
    if edge not in done and prob>0:
        print(edge,prob)

((1, 1, 1), (1, 1, 3)) 0.001
((1, 1, 0), (1, 1, 2)) 0.001
((1, 1, 2), (1, 2, 3)) 0.002
((1, 2, 1), (1, 2, 2)) 0.01
((1, 1, 1), (1, 2, 0)) 0.001
((1, 2, 2), (1, 2, 3)) 0.008
((1, 2, 0), (1, 2, 1)) 0.01
